In [1]:
import numpy as np
from scipy.io import loadmat
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import cm
import matplotlib as mpl
import cv2
import computer_vision as cv
from icecream import ic

# %matplotlib inline
%matplotlib qt
%config InlineBackend.figure_format = 'retina'
from matplotlib import rc
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)

In [2]:
def plot_lines_points_and_image(img_pts, img, l, path, plt_img=False, save=False):

    fig = plt.figure(figsize=(8,6))
    ax = plt.axes()
    
    cv.compute_and_plot_lines(l, img, ax)
    ax.plot(img_pts[0], img_pts[1], 'o', color='blue', label='Random points')

    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    # plt.gca().invert_yaxis()
    # ax.invert_yaxis()
    ax.set_aspect('equal')
    ax.legend(loc="upper right")
    fig.tight_layout()

    if plt_img:
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    
    # ax.view_init(elev=-20, azim=-120, roll=60)
    if save:
        fig.savefig(path, dpi=300)
    plt.show()


In [3]:
def plot_histogram(data, path, save=False):
    fig = plt.figure()
    plt.hist(data, bins=100, color='tab:blue')
    plt.xlabel('Error')
    plt.ylabel('Frequency')
    plt.xlim([0,6.5])
    fig.tight_layout()
    if save:
        fig.savefig(path, dpi=300)
    plt.show()

In [4]:
def get_skew_vector(T):
    t = np.array([T[2,1], T[0,2], T[1,0]])
    return t

In [10]:
def extract_P_from_E(E):

    U, S, VT = np.linalg.svd(E, full_matrices=False)
    # print(U @ np.diag([1,1,0] @ VT))
    print(S)

    if np.linalg.det(U @ VT) < 0:
        VT = -VT

    W = np.array([[0,1,0],[-1,0,0],[0,0,1]])
    Z = np.array([[0,-1,0],[1,0,0],[0,0,0]])

    S1 = U @ Z @ U.T
    S2 = U @ Z.T @ U.T

    R1 = U @ W @ VT
    R2 = U @ W.T @ VT

    t1 = get_skew_vector(S1)
    t2 = get_skew_vector(S2)

    P1 = np.concatenate((R1, t1[:, np.newaxis]), 1)
    P2 = np.concatenate((R1, t2[:, np.newaxis]), 1)
    P3 = np.concatenate((R2, t1[:, np.newaxis]), 1)
    P4 = np.concatenate((R2, t2[:, np.newaxis]), 1)

    P_arr = np.array([P1, P2, P3, P4])
    return P_arr

In [6]:
def get_default_camera():
    P = np.concatenate((np.eye(3), np.zeros(3)[:,np.newaxis]), 1)
    return P

In [7]:
def plot_cameras_and_3D_points(X, C_arr, axis_arr, s, path, save=False):
    
    fig = plt.figure(figsize=(8,6))
    ax = plt.axes(projection='3d')

    ax.plot(X[0], X[1], X[2], '.', ms=0.9, color='magenta', label='3D points')
    cv.plot_cameras_and_axes(ax, C_arr, axis_arr, s)

    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_zlabel('$z$')
    ax.axis('equal')

    ax.view_init(elev=-50, azim=-104, roll=60)

    plt.legend(loc="lower right")
    if save:
        fig.savefig(path, dpi=300)
    plt.show()

In [37]:
path = r'C:\Users\erikn\skola\EEN020-Computer-Vision\assignment-3'
data = r'\A3data\data'
compex1 = r'\compEx1data.mat'
compex2 = r'\compEx2data.mat'
img1 = r'\kronan1.jpg'
img2 = r'\kronan2.jpg'
report = r'\report-images'

kronan1 = cv.load_image(path+data+img1)
kronan2 = cv.load_image(path+data+img2)
K = cv.convert_mat_to_np(path+data+compex2, 'K')
K_inv = np.linalg.inv(K)
x = cv.convert_mat_to_np(path+data+compex1, 'x')
x1 = cv.dehomogenize(x[0,0])
x2 = cv.dehomogenize(x[1,0])

plt_pts = True
plt_hist = True
plt_3D = False
save = True

x1_norm = cv.transform_and_dehomogenize(K_inv, x1)
x2_norm = cv.transform_and_dehomogenize(K_inv, x2)

cv.check_mean_and_std(x1_norm)
cv.check_mean_and_std(x2_norm)

E = cv.estimate_E_DLT(x1_norm, x2_norm, print_svd=False)
F = cv.convert_E_to_F(E, K, K)

print('\nE:', E)
print('F:', F)

P_arr = extract_P_from_E(E)
P1 = get_default_camera()

X_arr_dh = np.array([cv.dehomogenize(cv.triangulate_3D_point_DLT(P1, P, x1_norm, x2_norm, print_svd=False)) for P in P_arr])

x1_arr = np.array([cv.transform(P1, X) for X in X_arr_dh])
valid_coords_P1 = np.array([np.sum(x[-1] > 0) for x in x1_arr]) > 0

x2_arr = np.array([cv.transform(P_arr[i], X_arr_dh[i]) for i in range(np.size(P_arr, 0))])
valid_coords_P2 = np.array([np.sum(x[-1] > 0) for x in x2_arr]) > 0

valid_coords = valid_coords_P1 * valid_coords_P2
X_valid = X_arr_dh[valid_coords][0]
P2_valid = P_arr[valid_coords][0]

# X_valid = X_arr_dh[0]
# P2_valid = P_arr[0]

P1 = cv.transform(K, P1)
P2 = cv.transform(K, P2_valid)

x1_proj = cv.transform_and_dehomogenize(P1, X_valid)
x2_proj = cv.transform_and_dehomogenize(P2, X_valid)

np.random.seed(0)
rand_mask = np.random.choice(np.size(x2_proj,1), 20, replace=False)
x1_proj_rand = x1_proj[:,rand_mask]
x2_proj_rand = x2_proj[:,rand_mask]
l1, l2 = cv.compute_epipolar_lines(F, x1_proj_rand, x2_proj_rand)

path1 = path+report+'/CE3_kronan1.png'
path2 = path+report+'/CE3_kronan2.png'

if plt_pts:
    plot_lines_points_and_image(x1_proj_rand, kronan1, l1, path1, plt_img=True, save=save)
    plot_lines_points_and_image(x2_proj_rand, kronan2, l2, path2, plt_img=True, save=save)

D1, D2, D_tot = cv.compute_epipolar_errors(F, x1, x2)

print('\nMean distance 1:', np.mean(D1))
print('Mean distance 2:', np.mean(D2))
print('Mean distance tot:', np.mean(D_tot))

path1 = path+report+'/CE3_hist_1.png'
path2 = path+report+'/CE3_hist_2.png'
path3 = path+report+'/CE3_hist_tot.png'

if plt_hist:
    plot_histogram(D1, path1)
    plot_histogram(D2, path2)
    plot_histogram(D_tot, path3)

P_arr = np.array([P1, P2])
C_arr, axis_arr = cv.compute_camera_and_normalized_principal_axis(P_arr, multi=True)

s = 10
path1 = path+report+'/CE3_3D_test.png'

if plt_3D:
    plot_cameras_and_3D_points(X_valid, C_arr, axis_arr, s, path, save=save)


x_mean: -0.03388656451837208 
x_std: 0.17551215010528629 
y_mean: 0.016434083846625286 
y_std: 0.13885331387003014

x_mean: -0.07181770613872099 
x_std: 0.18966030199829914 
y_mean: 0.01497495940066028 
y_std: 0.1443538259923785

E: [[-8.88845452e+00 -1.00580666e+03  3.77078254e+02]
 [ 1.25252308e+03  7.83677160e+01 -2.44817426e+03]
 [-4.72788839e+02  2.55019170e+03  1.00000000e+00]]
F: [[-1.55094115e-06 -1.75197697e-04  2.69029511e-01]
 [ 2.18172307e-04  1.36268632e-05 -1.23285416e+00]
 [-3.33116972e-01  1.21820346e+00  4.07542696e+01]]
[2.77933758e+03 2.77933758e+03 1.31996996e-13]

Mean distance 1: 1.9769590042006457
Mean distance 2: 2.08375320269186
Mean distance tot: 2.0303561034462527
